In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 821.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [5]:
from datasets import load_dataset
import cassio

In [6]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [7]:
from PyPDF2 import PdfReader

In [18]:
ASTRA_DB_ID = ""
ASTRA_DB_APPLICATION_TOKEN = ""
OPEN_AI_KEY = ""

In [9]:
pdf_reader = PdfReader("union_budget.pdf")

In [11]:
raw_text = ""
for i,pages in enumerate(pdf_reader.pages):
  content = pages.extract_text()
  if content:
    raw_text += content


In [19]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(133410205847792) fcf0f1b0-a96b-4569-acfa-462850f2bf5e-us-east1.db.astra.datastax.com:29042:bd11484a-5051-48b3-8b81-c834bf62bad5> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [20]:
llm = OpenAI(openai_api_key = OPEN_AI_KEY)
embeddings = OpenAIEmbeddings(openai_api_key = OPEN_AI_KEY)

In [21]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="qa_pdf",
    session=None,
    keyspace=None,
)

In [22]:
from langchain.text_splitter import CharacterTextSplitter

In [23]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [24]:

astra_vector_store.add_texts(texts)

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [25]:
query_text = "what are policy Highlights"

In [26]:
answer = astra_vector_index.query(query_text, llm=llm).strip()
print("ANSWER: \"%s\"\n" % answer)

ANSWER: "Some policy highlights mentioned in the context are the implementation of major economic railway corridor programs, the construction of additional houses under Pradhan Mantri Awas Yojana-Gram een, the extension of healthcare cover under Ayushman Bharat scheme, and the installation of coal gasification and liquefaction capacity."

